# 7장 - 집계 시 지저분한 데이터 다루기
1. 데이터 분석 프로젝트 대부분에서 집계가 필요. 가장 중요한 단계

## itertuples을 활용한 데이터 순회(안티 패턴)

In [1]:
import pandas as pd
import numpy as np

coviddaily=pd.read_csv('C:/data-cleansing-main/Chapter07/data/coviddaily720.csv',parse_dates=['casedate'])
ltbrazil=pd.read_csv('C:/data-cleansing-main/Chapter07/data/ltbrazil.csv')

In [4]:
coviddaily

,iso_code,casedate,location,continent,new_cases,new_deaths,population,pop_density,median_age,gdp_per_capita,hosp_beds,region
0,AFG,2019-12-31,Afghanistan,Asia,0.0,0.0,38928341.0,54.422,18.6,1803.987,0.5,South Asia
1,AFG,2020-01-01,Afghanistan,Asia,0.0,0.0,38928341.0,54.422,18.6,1803.987,0.5,South Asia
2,AFG,2020-01-02,Afghanistan,Asia,0.0,0.0,38928341.0,54.422,18.6,1803.987,0.5,South Asia
3,AFG,2020-01-03,Afghanistan,Asia,0.0,0.0,38928341.0,54.422,18.6,1803.987,0.5,South Asia
4,AFG,2020-01-04,Afghanistan,Asia,0.0,0.0,38928341.0,54.422,18.6,1803.987,0.5,South Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
29208,ZWE,2020-07-08,Zimbabwe,Africa,53.0,0.0,14862927.0,42.729,19.6,1899.775,1.7,Southern Africa
29209,ZWE,2020-07-09,Zimbabwe,Africa,98.0,0.0,14862927.0,42.729,19.6,1899.775,1.7,Southern Africa
29210,ZWE,2020-07-10,Zimbabwe,Africa,41.0,3.0,14862927.0,42.729,19.6,1899.775,1.7,Southern Africa
29211,ZWE,2020-07-11,Zimbabwe,Africa,16.0,1.0,14862927.0,42.729,19.6,1899.775,1.7,Southern Africa


In [3]:
coviddaily=coviddaily.sort_values(['location','casedate'])          # 데이터를 위치와 날짜별로 정렬

### itertuples로 행들을 순회(?)

In [7]:
prevloc='ZZZ'
rowlist=[]
for row in coviddaily.itertuples():
  if (prevloc != row.location):
    if (prevloc != 'ZZZ'):
      rowlist.append({'location':prevloc, 'casecnt':casecnt})
    casecnt = 0
    prevloc = row.location
  casecnt += row.new_cases

rowlist.append({'location':prevloc,'casecnt':casecnt})
len(rowlist)
rowlist

[{'location': 'Afghanistan', 'casecnt': 34451.0},
 {'location': 'Albania', 'casecnt': 3371.0},
 {'location': 'Algeria', 'casecnt': 18712.0},
 {'location': 'Andorra', 'casecnt': 855.0},
 {'location': 'Angola', 'casecnt': 483.0},
 {'location': 'Anguilla', 'casecnt': 3.0},
 {'location': 'Antigua and Barbuda', 'casecnt': 74.0},
 {'location': 'Argentina', 'casecnt': 97509.0},
 {'location': 'Armenia', 'casecnt': 31392.0},
 {'location': 'Aruba', 'casecnt': 105.0},
 {'location': 'Australia', 'casecnt': 9553.0},
 {'location': 'Austria', 'casecnt': 18795.0},
 {'location': 'Azerbaijan', 'casecnt': 23521.0},
 {'location': 'Bahamas', 'casecnt': 111.0},
 {'location': 'Bahrain', 'casecnt': 32470.0},
 {'location': 'Bangladesh', 'casecnt': 181129.0},
 {'location': 'Barbados', 'casecnt': 103.0},
 {'location': 'Belarus', 'casecnt': 64767.0},
 {'location': 'Belgium', 'casecnt': 62606.0},
 {'location': 'Belize', 'casecnt': 37.0},
 {'location': 'Benin', 'casecnt': 1587.0},
 {'location': 'Bermuda', 'casecnt'

In [11]:
# 소계를 담은 rowlist로부터 데이터프레임 생성
covidtotals=pd.DataFrame(rowlist)
covidtotals.head()

,location,casecnt
0,Afghanistan,34451.0
2,Algeria,18712.0
1,Albania,3371.0
3,Andorra,855.0
4,Angola,483.0


In [19]:
ltbrazil=ltbrazil.sort_values(['station','month'])      # 지표온도 데이터 정렬
ltbrazil=ltbrazil.dropna(subset=['temperature'])                 # 온도에 누락값이 있는 행 드롭
ltbrazil

,locationid,year,month,temperature,latitude,longitude,elevation,station,countryid,country,latabs
648,BR000352000,2019,8,28.55,-3.200,-52.200,112.0,ALTAMIRA,BR,Brazil,3.200
740,BR000352000,2019,9,28.85,-3.200,-52.200,112.0,ALTAMIRA,BR,Brazil,3.200
832,BR000352000,2019,10,28.65,-3.200,-52.200,112.0,ALTAMIRA,BR,Brazil,3.200
924,BR000352000,2019,11,28.00,-3.200,-52.200,112.0,ALTAMIRA,BR,Brazil,3.200
1016,BR000352000,2019,12,27.50,-3.200,-52.200,112.0,ALTAMIRA,BR,Brazil,3.200
...,...,...,...,...,...,...,...,...,...,...,...
662,BR00B6-0360,2019,8,24.05,-20.417,-49.983,503.0,VOTUPORANGA,BR,Brazil,20.417
754,BR00B6-0360,2019,9,26.45,-20.417,-49.983,503.0,VOTUPORANGA,BR,Brazil,20.417
846,BR00B6-0360,2019,10,27.00,-20.417,-49.983,503.0,VOTUPORANGA,BR,Brazil,20.417
938,BR00B6-0360,2019,11,25.15,-20.417,-49.983,503.0,VOTUPORANGA,BR,Brazil,20.417


In [20]:
# 한 기간과 그 다음 기간 사이에 큰 변화가 있는 행 제외
## 그 해의 평균 기온 계산, 이전 달 기온보다 3도 높거나 낮은 값 제외
prevstation = 'ZZZ'
prevtemp = 0
rowlist = []

for row in ltbrazil.itertuples():
  if (prevstation!=row.station):
    if (prevstation!='ZZZ'):
      rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt,
                      'stationcnt':stationcnt})
    tempcnt = 0
    stationcnt = 0
    prevstation = row.station
  # choose only rows that are within 3 degrees of the previous temperature  
  if ((0 <= abs(row.temperature-prevtemp) <= 3) or (stationcnt==0)):
    tempcnt += row.temperature
    stationcnt += 1
  prevtemp = row.temperature

rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})
rowlist[0:5]
ltbrazilavgs = pd.DataFrame(rowlist)
ltbrazilavgs.head()

,station,avgtemp,stationcnt
0,ALTAMIRA,28.310000,5
1,ALTA_FLORESTA_AERO,29.433636,11
2,ARAXA,21.612500,4
3,BACABAL,29.750000,4
4,BAGE,20.366667,9


In [21]:
for i in ltbrazil.itertuples():
    print(i.temperature)

28.55
28.85
28.65
28.0
27.5
26.95
28.01
28.12
28.8
29.58
30.37
29.92
31.65
32.69
28.72
28.88
28.8
24.1
22.45
20.5
19.4
29.7
29.65
29.8
29.85
25.1
23.9
21.8
20.15
16.9
17.45
12.4
13.5
15.75
18.95
21.9
23.75
27.9
27.85
26.9
26.45
26.2
25.3
24.15
27.9
28.5
29.15
28.9
28.3
29.05
29.15
28.4
27.45
27.55
27.2
28.0
29.15
28.4
28.6
28.0
28.25
27.9
27.6
28.3
28.15
27.55
30.65
30.7
29.8
27.45
26.9
26.9
26.95
26.35
24.55
23.65
24.95
27.55
28.9
29.2
28.2
29.3
29.8
28.25
27.1
25.55
23.95
22.05
21.95
23.4
26.22
25.85
26.7
27.57
25.65
25.21
24.95
22.98
21.98
20.24
21.63
24.69
26.34
25.15
25.49
27.6
27.2
27.35
27.9
28.4
28.5
28.8
28.85
28.9
28.5
28.7
28.55
24.55
23.25
23.55
22.9
21.2
19.15
18.4
20.15
22.8
23.7
23.5
23.55
25.9
26.1
26.15
26.35
26.55
26.7
27.25
27.75
28.8
28.0
27.55
27.3
26.55
27.8
27.95
27.55
27.75
27.0
27.6
28.35
27.65
27.3
26.9
30.3
30.65
32.0
28.45
28.15
28.4
28.45
29.4
30.25
29.7
29.55
29.7
29.5
28.95
28.35
28.45
26.4
25.65
26.6
29.15
30.6
31.05
30.65
23.55
23.5
22.9
22.7
21.95
19.8

## 정리
1. 표 데이터를 다룰 때는 여러 행에 걸쳐 합계를 계산하는 작업이 가장 어렵다. 어떤 언어를 사용하든 복잡한 일
2. 어쨌든 해내야 된다는 뜻인 듯..ㅋㅋ